Importing all Libraries needed to train as well as get information from the data base

In [28]:
import pandas as pd
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pytds
import re #For Preprocessing
import spacy #More Preprocessing
from time import time #To Time the Operations


Connecting to database to retrieve student and training resumes

In [29]:
connect_string = ""
try:
    con = pytds.connect(server = '34.69.166.47', database = 'TBDraft', user = 'sqlserver', password='TBuilder')
    cursor = con.cursor()
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

INFO - 22:31:00: Opening socket to 34.69.166.47:1433
INFO - 22:31:00: Sending PRELOGIN lib_ver=10b0000 enc_flag=2 inst_name=b'MSSQLServer' mars=False
INFO - 22:31:00: Got PRELOGIN response crypt=2 mars=0
INFO - 22:31:00: Sending LOGIN tds_ver=74000004 bufsz=4096 pid=3784 opt1=f0 opt2=2 opt3=8 cli_tz=-360 cli_lcid=1033 cli_host=DESKTOP-02AVBJA lang= db=TBDraft
INFO - 22:31:00: switched to database TBDraft
INFO - 22:31:00: switched collation to Collation(lcid=1033, sort_id=52, ignore_case=True, ignore_accent=False, ignore_width=True, ignore_kana=True, binary=False, binary2=False, version=0)
INFO - 22:31:00: switched language to us_english
INFO - 22:31:00: Got LOGINACK tds_ver=74000004 srv_name=Microsoft SQL Server   srv_ver=f001052
INFO - 22:31:00: Sending BEGIN TRAN il=0


 is working


In [30]:
##After connecting to database extract preprocessed information
#These are the base training resumes
df_training = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", con)
#There are the processed student resumes
#df_students = pd.read_sql_query("SELECT ALL * FROM StudentInformation", conn)
con.close()
print(df_training)
#print(df_students)


INFO - 22:31:00: Sending query SELECT ALL * FROM TrainingResumes


                  keyword                                           dataText
0    Electronics_Engineer    andre salazar fort worth tx email talent ele...
1    Electronics_Engineer    arturo cervante mission tx email seek employ...
2    Electronics_Engineer    juan avila charleston sc email experience el...
3    Electronics_Engineer    travis ray engineer product designer develop...
4        Embedded_Systems    daniel koh forney tx email software developm...
..                    ...                                                ...
101     Software_Engineer    kai schmidt software engineer citrus height ...
102     Software_Engineer    nathan nguyen irvine email work experience s...
103     Software_Engineer    peter pagano bohemia ny email authorize work...
104     Software_Engineer    steven nichol c c software engineer grand ra...
105     Software_Engineer    xueqiang pan highland ut email skilled innov...

[106 rows x 2 columns]


In [31]:
df = pd.DataFrame(columns=['Keyword', 'Data'])
df['Keyword'] = df_training['keyword'] #+ df_students['']
df['Data'] = df_training['dataText'] #+ df_students['']
df

,Keyword,Data
0,Electronics_Engineer,andre salazar fort worth tx email talent ele...
1,Electronics_Engineer,arturo cervante mission tx email seek employ...
2,Electronics_Engineer,juan avila charleston sc email experience el...
3,Electronics_Engineer,travis ray engineer product designer develop...
4,Embedded_Systems,daniel koh forney tx email software developm...
...,...,...
101,Software_Engineer,kai schmidt software engineer citrus height ...
102,Software_Engineer,nathan nguyen irvine email work experience s...
103,Software_Engineer,peter pagano bohemia ny email authorize work...
104,Software_Engineer,steven nichol c c software engineer grand ra...


For Now we use tika to parse resumes and clean them

In [32]:
sent = [row.split() for row in df['Data']] #Change change ['clean'] to whatever we actually label the columns as
documents = [TaggedDocument(doc, tags=[df.Keyword[i]]) for i, doc in enumerate(sent)]

Training the model and saving it

In [33]:
threads = multiprocessing.cpu_count()
model = Doc2Vec(documents, 
                dm = 0,
                dbow_words=1, 
                alpha=0.03, 
                vector_size=30, 
                min_alpha = 0.0007, 
                min_count = 30, 
                negative=15,
                workers=threads-1)
model.save('d2v_model')

INFO - 22:31:02: collecting all words and their counts
INFO - 22:31:02: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 22:31:02: collected 7058 word types and 6 unique tags from a corpus of 106 examples and 56454 words
INFO - 22:31:02: Creating a fresh vocabulary
INFO - 22:31:02: Doc2Vec lifecycle event {'msg': 'effective_min_count=30 retains 369 unique words (5.2281099461603855%% of original 7058, drops 6689)', 'datetime': '2021-11-24T22:31:02.276661', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 22:31:02: Doc2Vec lifecycle event {'msg': 'effective_min_count=30 leaves 31457 word corpus (55.72147234916923%% of original 56454, drops 24997)', 'datetime': '2021-11-24T22:31:02.277654', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': '

In [37]:
tokens ="software".split()
new_vec = model.infer_vector(tokens)
sims = model.dv.most_similar(positive=[new_vec])
sims


[('RF_Analog_Engineer', 0.6601521968841553),
 ('Embedded_Systems', 0.6389424204826355),
 ('Electronics_Engineer', 0.6247180700302124),
 ('Power_Systems_Engineer', 0.612964928150177),
 ('Machine_Learning', 0.5309510231018066),
 ('Software_Engineer', 0.5123679041862488)]